# Importations

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import sys
import os
import cv2

# Coordinates of first and last point of the line

In [5]:

#lat_a = (35,48, 36.95)  
#lon_a = (10, 36, 27.88) 
#lat_b = (35, 48, 17.11)  
#lon_b = (10, 35,2.83)
lat_a = (35, 49, 43.24)  
lon_a = (10, 36, 1.91)
lat_b=(35, 50, 36.53)
lon_b=(10, 36, 3.71)


# Linear Interpolation

In [6]:

def dms_to_dd(degrees, minutes, seconds):
    return degrees + minutes / 60 + seconds / 3600
lat_a_dd = dms_to_dd(*lat_a)
lon_a_dd = dms_to_dd(*lon_a)
lat_b_dd = dms_to_dd(*lat_b)
lon_b_dd = dms_to_dd(*lon_b)
path = 'E:/validation'
image_files = [f for f in os.listdir(path) if f.endswith('.jpg')]

interpolated_coords = []
for i, image_name in enumerate(image_files):
    ratio = i / (len(image_files) - 1)
    lat = lat_a_dd + (lat_b_dd - lat_a_dd) * ratio
    lon = lon_a_dd + (lon_b_dd - lon_a_dd) * ratio
    interpolated_coords.append(( lat, lon))


print(interpolated_coords)

[(35.828677777777784, 10.600530555555554), (35.82869280597858, 10.600531063169768), (35.82870783417936, 10.60053157078398), (35.828722862380154, 10.600532078398194), (35.828737890580946, 10.600532586012408), (35.82875291878173, 10.60053309362662), (35.82876794698252, 10.600533601240834), (35.82878297518331, 10.600534108855047), (35.8287980033841, 10.60053461646926), (35.82881303158489, 10.600535124083473), (35.82882805978568, 10.600535631697687), (35.82884308798647, 10.600536139311899), (35.82885811618726, 10.600536646926113), (35.82887314438805, 10.600537154540326), (35.82888817258884, 10.600537662154538), (35.828903200789625, 10.600538169768752), (35.82891822899042, 10.600538677382966), (35.82893325719121, 10.600539184997178), (35.828948285391995, 10.600539692611392), (35.82896331359279, 10.600540200225606), (35.82897834179358, 10.60054070783982), (35.828993369994365, 10.600541215454031), (35.82900839819516, 10.600541723068245), (35.82902342639595, 10.600542230682459), (35.8290384545

# Decimal to Degree

In [7]:

def dd_to_dms(decimal_degrees):
    degrees = int(decimal_degrees)
    minutes = int((decimal_degrees - degrees) * 60)
    seconds = (decimal_degrees - degrees - minutes / 60) * 3600
    return degrees, minutes, seconds



# Save GPS Coordinates 

In [8]:
#data_dir = "C:/Users/ahmed/Desktop/test"
data_dir = 'E:/'
image_files = os.listdir(path)
image_coords_list = []
for image_file, coord in zip(image_files, interpolated_coords):
    lat_deg, lat_min, lat_sec = dd_to_dms(coord[0])
    lon_deg, lon_min, lon_sec = dd_to_dms(coord[1])
    lat_str = f"{lat_deg}°{lat_min}'{lat_sec:.2f}\"N"
    lon_str = f"{lon_deg}°{lon_min}'{lon_sec:.2f}\"E"

    image_coords_list.append((image_file, lat_str, lon_str))


csv_filename = "image_coordinates_vald.csv"
csv_filepath = os.path.join(data_dir, csv_filename)
with open(csv_filepath, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Image Name", "Latitude", "Longitude"])
    for image_coord in image_coords_list:
        writer.writerow(image_coord)

print(f"CSV file '{csv_filepath}' has been created successfully.")

CSV file 'E:/image_coordinates_vald.csv' has been created successfully.


In [9]:
df = pd.read_csv("E:/image_coordinates.csv")

In [10]:
df.head(5)

,Image Name,Latitude,Longitude
0,data-000000.jpg,"35°52'19.05""N","10°36'10.28""E"
1,data-000001.jpg,"35°52'18.89""N","10°36'10.35""E"
2,data-000002.jpg,"35°52'18.73""N","10°36'10.43""E"
3,data-000003.jpg,"35°52'18.57""N","10°36'10.50""E"
4,data-000004.jpg,"35°52'18.41""N","10°36'10.58""E"


In [11]:
df1=pd.read_csv("E:/image_coordinates_vald.csv")
df1.sample(10)


,Image Name,Latitude,Longitude
713,data-000713.jpg,"35°50'21.81""N","10°36'3.21""E"
160,data-000160.jpg,"35°49'51.90""N","10°36'2.20""E"
500,data-000500.jpg,"35°50'10.29""N","10°36'2.82""E"
141,data-000141.jpg,"35°49'50.87""N","10°36'2.17""E"
182,data-000182.jpg,"35°49'53.09""N","10°36'2.24""E"
216,data-000216.jpg,"35°49'54.93""N","10°36'2.30""E"
144,data-000144.jpg,"35°49'51.03""N","10°36'2.17""E"
829,data-000829.jpg,"35°50'28.09""N","10°36'3.42""E"
38,data-000038.jpg,"35°49'45.30""N","10°36'1.98""E"
352,data-000352.jpg,"35°50'2.28""N","10°36'2.55""E"


# Data Augmentation

In [23]:

def process_images(csv_filepath, df, image_files, path, transform_type,output_path):
    
    output_dir = os.path.join(output_path, transform_type)
    os.makedirs(output_dir, exist_ok=True)

   
    csv_filename = f"{transform_type}_data.csv"
    csv_filepath = os.path.join(output_dir, csv_filename)
    with open(csv_filepath, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)

        
        for i, image in enumerate(image_files):       
            a = df['Image Name'][i]
            a = a.split('-')
            if transform_type == 'grayscale':
                a = a[0] + 'grayscale-' + a[1]
            elif transform_type == 'resize':
                a = a[0] + 'resize-' + a[1]
            elif transform_type == 'flip':
                a = a[0] + 'flip-' + a[1]
            elif transform_type == 'rotate':
                a = a[0] + 'rotate-' + a[1]
            elif transform_type == 'crop':
                a = a[0] + 'crop-' + a[1]
            elif transform_type == 'gauss':
                a = a[0] + 'gauss-' + a[1]

            b = df['Latitude'][i]
            c = df['Longitude'][i]

            
            csv_writer.writerow([a, b, c])

            
            image_path = os.path.join(path, image)
            if not os.path.isfile(image_path):
                print(f"Error: Image file not found - {image_path}")
                continue

            img = cv2.imread(image_path)
            if img is None:
                print(f"Error: Failed to load image - {image_path}")
                continue

            if transform_type == 'grayscale':
                grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                cv2.imwrite(os.path.join(output_dir, a), grayscale)
            elif transform_type == 'resize':
                resized = cv2.resize(img, (128, 128))
                cv2.imwrite(os.path.join(output_dir, a), resized)
            elif transform_type == 'flip':
                flipped = cv2.flip(img, 1)  
                cv2.imwrite(os.path.join(output_dir, a), flipped)
            elif transform_type == 'rotate':
                rotated = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)  
                cv2.imwrite(os.path.join(output_dir, a), rotated)
            elif transform_type == 'crop':
                cropped = img[50:250, 50:250]  
                cv2.imwrite(os.path.join(output_dir, a), cropped)
            elif transform_type == 'gauss':
                gaussian_blur = cv2.GaussianBlur(img, (5, 5), 5)  
                

                cv2.imwrite(os.path.join(output_dir, a), gaussian_blur)


In [24]:
process_images(csv_filepath, df, image_files, path, 'gauss','E:/Augmentedvalidation')